In [15]:
# 自动求导计算一个函数在指定值上的导数
# 计算图：将代码分解成操作子，将计算表示成一个无环图
# 自动求导的两种模式：
# ①正向累积：需要存储正向的所有结果，内存复杂度O(n)；②反向传播，又称反向传递，内存复杂度O(1)，后者更常用
import torch

'''求梯度就是求导数'''

In [16]:
# 假设我们想对函数y = 2 * xT * x求关于列向量x求导
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [17]:
# 在我们计算y关于的梯度之前，我们需要一个地方来存储梯度
x.requires_grad_(True)  # 等价于x = torch.arange(4.0, requires_grad=True)
# 这意味着在后续的计算中，PyTorch 将会跟踪所有对 x 的操作，以便在反向传播时自动计算梯度
# 在计算梯度之前，x.grad 默认是 None，在反向传播计算梯度之后，x.grad 将会存储 x 的梯度

tensor([0., 1., 2., 3.], requires_grad=True)

In [18]:
# 现在让我们计算y(x和x内积乘以2), z(2y), h(z的平方)
y = 2 * torch.dot(x, x)
y.retain_grad()  # 保留y的梯度
z = 2 * y
z.retain_grad()  # 保留z的梯度
h = z ** 2
y, z, h

(tensor(28., grad_fn=<MulBackward0>),
 tensor(56., grad_fn=<MulBackward0>),
 tensor(3136., grad_fn=<PowBackward0>))

In [19]:
# 调用反向传播函数
h.backward()  # 求导
z.grad, y.grad, x.grad  # 求完之后就可以通过_.grad来访问导数了
# z.grad 中，dh/dz = 2 * z, 而z = 2y = 56, 所以z.grad = 2 * 56 = 112
# y.grad 中，dz/dy = 2, 所以y.grad = z.grad * dz/dy = 112 * 2 = 224
# x.grad 中，dy/dx = 4x, 所以x.grad = y.grad * dy/dx = 224 * 4x = [0, 896, 1792, 2688]

(tensor(112.), tensor(224.), tensor([   0.,  896., 1792., 2688.]))

In [20]:
z.grad == 2 * z, y.grad == z.grad * 2, x.grad == y.grad * 4 * x

(tensor(True), tensor(True), tensor([True, True, True, True]))

In [21]:
# 在默认情况下，PyTorch 会累积梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

In [22]:
# 对非标量调用`backward`需要传入一个`gradient`参数，该参数指定微分函数关于`self`的梯度。
# 在我们的例子中，我们只想求导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()  # 这里y.sum()是标量, 此时y.sum() = x0^3 + x1^3 + x2^3 + x3^3, 所以y.sum().backward() = dy/dx = [3x0^2, 3x1^2, 3x2^2, 3x3^2]
x, x * x, x.grad
# 深度学习中一般对损失函数求导，而损失函数是标量，所以一般都是对标量调用backward

(tensor([0., 1., 2., 3.], requires_grad=True),
 tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>),
 tensor([ 0.,  3., 12., 27.]))

In [23]:
# 将某些计算移动到记录的计算图之外
x.grad.zero_()
y = x * x
u = y.detach()  # 将y的计算结果当成一个常数，赋值给u，此时u不再是关于x的函数，而是一个常数, u = [0, 1, 4, 9]
z = u * x 
z.sum().backward()  # dz/dx = u = [0, 1, 4, 9]
x.grad == u

tensor([True, True, True, True])

In [24]:
# 作个简单对比
x.grad.zero_()
y = x * x
y.sum().backward()  # dy/dx = 2x
x.grad == 2 * x

tensor([True, True, True, True])

In [25]:
# 即使构建函数的计算图需要通过 Python 控制流（如条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()
a, a.grad == d / a
# 在函数 f 中，b 是通过不断乘以 2 得到的，直到其范数大于等于 1000。
# 假设 a 是一个标量，经过若干次乘以 2 后，b 可以表示为 b = a * 2^n，其中 n 是使得 b.norm() >= 1000 的最小整数
# 在c = b的情况下，d = f(a) = c = a * 2^n
# 在c = 100 * b的情况下，d = f(a) = c = 100 * a * 2^n
# 无论哪种情况，d = f(a) = c都是a的线性函数，所以df(a)/da = 2^n或者2^n * 100 = d / a

(tensor(1.4223, requires_grad=True), tensor(True))

In [26]:
# 再来看看中断梯度追踪
x = torch.tensor(1.0, requires_grad=True)
y1 = x ** 2 
with torch.no_grad():  # 上下文管理器，不追踪梯度，常用于测试集
    y2 = x ** 3
y3 = y1 + y2

print(x.requires_grad)
print(y1, y1.requires_grad) # True
print(y2, y2.requires_grad) # False
print(y3, y3.requires_grad) # True
# 可以看到，上面的y2是没有grad_fn而且y2.requires_grad=False的，而y3是有grad_fn的，y3.requires_grad=True

True
tensor(1., grad_fn=<PowBackward0>) True
tensor(1.) False
tensor(2., grad_fn=<AddBackward0>) True


In [27]:
y3.backward()
x.grad == 2 * x
# 由于 y2 的定义是被torch.no_grad():包裹的，所以与 y2 有关的梯度是不会回传的，
# 只有与 y1 有关的梯度才会回传，即 x^2 对 x 的梯度

tensor(True)

In [28]:
# 此外，如果我们想要修改tensor的数值，但是又不希望被autograd记录（即不会影响反向传播），那么我么可以对tensor.data进行操作
x = torch.ones(1,requires_grad=True)

print(x.data) # 还是一个tensor
print(x.data.requires_grad) # 但是已经是独立于计算图之外

y = 2 * x
x.data *= 100 # 只改变了值，不会记录在计算图，所以不会影响梯度传播

y.backward()
print(x) # 更改data的值也会影响tensor的值
print(x.grad)

tensor([1.])
False
tensor([100.], requires_grad=True)
tensor([2.])
